In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, mapping
from shapely.ops import unary_union, cascaded_union
from shapely import wkt
from urllib.request import urlopen, HTTPError
import glob
from functools import reduce
import requests
from io import StringIO

## Concating

In [2]:
lists = glob.glob('Polygons/*.geojson')

In [3]:
gdf1 = gpd.GeoDataFrame()
a = gpd.read_file(lists[0])
b = gpd.read_file('Polygons/subnational_201110_CS_ea.geojson')

gdf1 = gdf1.append(a)
gdf1 = gdf1.append(b)

In [4]:
gdf1.CS.value_counts()

1.0     242
2.0     104
3.0      60
88.0     38
4.0      28
5.0      10
99.0      6
Name: CS, dtype: int64

In [8]:
gdf_all = gpd.GeoDataFrame()
for i in lists:
    a = gpd.read_file(i)
    gdf_all = gdf_all.append(a)

In [13]:
gdf_all.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 51766 entries, 0 to 373
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   ADM1_CODE    51766 non-null  int64   
 1   ADM0_NAME    51766 non-null  object  
 2   Start        51766 non-null  object  
 3   End          51766 non-null  object  
 4   index_right  24760 non-null  float64 
 5   CS           24760 non-null  float64 
 6   HA0          20096 non-null  float64 
 7   HA           174 non-null    float64 
 8   geometry     51766 non-null  geometry
 9   CS201110_P   96 non-null     float64 
 10  CS201010_P   154 non-null    float64 
dtypes: float64(6), geometry(1), int64(1), object(3)
memory usage: 4.7+ MB


## Cleaning

In [19]:
gdf_new = gdf_all[['ADM1_CODE', 'ADM0_NAME', 'Start', 'End', 'CS', 'geometry']]

In [25]:
gdf_a.CS.value_counts()

1.0     11440
2.0      6678
3.0      3130
88.0     1672
99.0     1282
4.0       540
5.0        18
Name: CS, dtype: int64

In [33]:
gdf_a = gdf_new.dropna(subset=['CS']).reset_index(drop=True)
gdf_b = gdf_a[gdf_a.CS.isin([1,2,3,4,5])]
gdf_c = gdf_b.sort_values(by=['ADM1_CODE', 'Start', 'End', 'CS'], ascending = False)
gdf_d = gdf_c.drop_duplicates(subset=['ADM1_CODE', 'Start', 'End'], keep = 'first').reset_index(drop=False)

In [35]:
gdf_d.CS.value_counts()

1.0    3068
2.0    1498
3.0    1096
4.0     259
5.0       9
Name: CS, dtype: int64

In [36]:
gdf_d.to_file('HFIC_new.geojson', driver = 'GeoJSON')